IMPORT Library & Konfigurasi Path

In [1]:
# %% [markdown]
# # FASE 1: Eksplorasi dan Persiapan Data Wisata Jember
# 
# **Tujuan:**
# 1. Membaca dataset mentah (`destinasi_jember.csv`).
# 2. Melakukan analisis data eksplorasi (EDA) singkat.
# 3. Melakukan *feature engineering* dengan menggabungkan kolom teks.
# 4. Membersihkan teks fitur.
# 5. Menyimpan dataset yang sudah bersih dan siap pakai ke `data/processed/`.

# %%
# ======================================================
# 1️⃣ IMPORT LIBRARY & KONFIGURASI PATH
# ======================================================
import pandas as pd
import sys
from pathlib import Path
from tqdm import tqdm

# Menambahkan direktori utama proyek ke path Python
# Ini memastikan kita bisa mengimpor dari folder 'utils' dengan andal
BASE_DIR = Path().resolve().parent
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

from utils.text_preprocessing import clean_text

# Menginisialisasi tqdm untuk integrasi dengan pandas (menampilkan progress bar)
tqdm.pandas()

print("✅ Library dan path berhasil dikonfigurasi.")

✅ Library dan path berhasil dikonfigurasi.


Memuat Dataset Mentah

In [13]:
# %% [markdown]
# ## 2. Memuat Dataset Mentah
# 
# Membaca file `destinasi_jember.csv` dari folder `data/raw/` dengan metode yang paling andal.

# %%
# ======================================================
# 2️⃣ MEMUAT DATASET MENTAH (VERSI PALING ANDAL)
# ======================================================
RAW_DATA_PATH = BASE_DIR / "data" / "raw" / "destinasi_jember.csv"

try:
    # ⭐ PERBAIKAN UTAMA: Menggunakan engine='python' yang lebih andal untuk CSV kompleks
    # Mesin ini lebih pintar dalam menangani koma yang ada di dalam tanda kutip.
    df = pd.read_csv(
        RAW_DATA_PATH,
        engine='python'
    )

    # Menghapus kolom kosong jika ada (jaring pengaman)
    df = df.dropna(axis=1, how='all')

    print(f"✅ Dataset berhasil dimuat dari '{RAW_DATA_PATH.name}'")
    print(f"Jumlah baris: {len(df)}, Jumlah kolom: {len(df.columns)}")

    print("\n📋 Nama Kolom Dataset:")
    print(df.columns.tolist())

    print("\n🔍 5 Baris Pertama:")
    display(df.head()) # Menggunakan display() agar output di notebook lebih rapi

except Exception as e:
    print(f"❌ Gagal membaca dataset: {e}")

✅ Dataset berhasil dimuat dari 'destinasi_jember.csv'
Jumlah baris: 55, Jumlah kolom: 7

📋 Nama Kolom Dataset:
['id', 'nama_wisata', 'kategori', 'kota', 'alamat', 'deskripsi', 'gambar']

🔍 5 Baris Pertama:


,id,nama_wisata,kategori,kota,alamat,deskripsi,gambar
0,1,Puncak Rembangan Resort,Rekreasi,Jember,"Darungan, Kemuning Lor, Kec. Arjasa, Kabupaten...",Puncak Rembangan adalah destinasi wisata pegun...,assets/images/1.png
1,2,Pantai Watu Ulo,Pantai,Jember,"Sumberrejo, Kec. Ambulu, Kabupaten Jember, Jaw...",Pantai Watu Ulo terkenal karena batu karang pa...,assets/images/2.png
2,3,Puslit Kopi & Kakao Indonesia,Agrowisata,Jember,"Puslitkoka Renteng, Gebang, Nogosari, Kec. Ram...",Pusat Penelitian Kopi dan Kakao (Puslitkoka) i...,assets/images/3.png
3,4,Bin Cigar,Edukasi,Jember,"Jl. Brawijaya No.3, Krajan, Jubung, Kec. Sukor...",Bin Cigar adalah tempat edukasi/museum rokok/c...,assets/images/4.png
4,5,Makam Habib Soleh Tanggul,Religi,Jember,"Krajan, Tanggul Kulon, Kec. Tanggul, Kabupaten...",Makam Habib Soleh Tanggul adalah situs religi ...,assets/images/5.png


Analisis Data Eksplorasi (EDA) Singkat

In [ ]:
# %% [markdown]
# ## 3. Analisis Data Eksplorasi (EDA) Singkat
# 
# Memahami struktur data, tipe kolom, dan memeriksa data yang hilang.

# %%
# ======================================================
# 3️⃣ ANALISIS DATA EKSPLORASI (EDA)
# ======================================================
# Menampilkan informasi ringkas (tipe data, non-null counts)
print("Informasi Dataset:")
df.info()

print("\nJumlah Nilai Kosong per Kolom:")
print(df.isnull().sum())

Penanganan Nilai Kosong

In [ ]:
# %% [markdown]
# **Penanganan Nilai Kosong:**
# 
# Sebelum menggabungkan kolom teks, kita perlu memastikan tidak ada nilai `NaN` (kosong) yang akan menyebabkan error. Kita akan mengisi nilai kosong di kolom teks dengan string kosong.

# %%
# ======================================================
# 4️⃣ PENANGANAN NILAI KOSONG
# ======================================================
text_columns = ['nama_wisata', 'kategori', 'kota', 'alamat', 'deskripsi']
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].fillna('')

print("✅ Nilai kosong pada kolom teks sudah ditangani.")

Feature Engineering

In [ ]:
# %% [markdown]
# ## 5. Feature Engineering: Membuat Kolom "Fitur"
# 
# Ini adalah langkah inti. Kita akan membuat satu kolom teks super (`fitur`) yang menjadi "DNA" dari setiap destinasi wisata. Semakin kaya DNA ini, semakin baik model rekomendasi kita.

# %%
# ======================================================
# 5️⃣ FEATURE ENGINEERING
# ======================================================
# Menggabungkan semua kolom teks yang relevan.
# Kita menggandakan 'kategori' dan 'nama_wisata' untuk memberikan bobot lebih pada kata kunci ini.
df['fitur'] = (
    df['nama_wisata'] + ' ' + 
    df['kategori'] + ' ' + 
    df['kategori'] + ' ' + # Memberi bobot lebih pada kategori
    df['kota'] + ' ' + 
    df['alamat'] + ' ' + 
    df['deskripsi']
)

# Tampilkan hasilnya untuk melihat "DNA" yang baru dibuat
print("Contoh kolom 'fitur' yang baru dibuat:")
pd.set_option('display.max_colwidth', 200) # Agar teks tidak terpotong
df[['nama_wisata', 'fitur']].head()

Pembersihan Teks

In [ ]:
# %% [markdown]
# ## 6. Pembersihan Teks
# 
# Menerapkan fungsi `clean_text` dari `utils` pada "DNA" teks di kolom `fitur`.

# %%
# ======================================================
# 6️⃣ PEMBERSIHAN TEKS
# ======================================================
print("Memulai proses pembersihan teks (mungkin butuh beberapa saat)...")

# Menggunakan .progress_apply() dari tqdm untuk menampilkan progress bar.
df['fitur_bersih'] = df['fitur'].progress_apply(clean_text)

print("\nContoh hasil pembersihan teks:")
df[['nama_wisata', 'fitur_bersih']].head()

Menyimpan Dataset yang Sudah Diproses

In [ ]:
# %% [markdown]
# ## 7. Menyimpan Dataset yang Sudah Diproses
# 
# Menyimpan DataFrame yang sudah bersih ini ke folder `data/processed/` untuk digunakan di Fase 2 (Pemodelan).

# %%
# ======================================================
# 7️⃣ MENYIMPAN HASIL PROSES
# ======================================================
PROCESSED_DATA_PATH = BASE_DIR / "data" / "processed" / "destinasi_processed.csv"

# Membuat folder 'processed' jika belum ada
PROCESSED_DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

# Menyimpan ke CSV
df.to_csv(PROCESSED_DATA_PATH, index=False)

print(f"🎉 Dataset yang sudah bersih berhasil disimpan di: '{PROCESSED_DATA_PATH}'")

Validasi Akhir

In [ ]:
# %% [markdown]
# ## 8. Validasi Akhir
# 
# Memeriksa beberapa baris terakhir dari data yang sudah diproses untuk memastikan semuanya terlihat benar.

# %%
# ======================================================
# 8️⃣ VALIDASI AKHIR
# ======================================================
# Tampilkan 5 baris terakhir dengan kolom-kolom kunci
print("Pratinjau 5 data terakhir dari file yang sudah diproses:")
df[['id', 'nama_wisata', 'fitur_bersih', 'gambar']].tail()